In [5]:
import pandas as pd
from pymongo import MongoClient
from bson import json_util

# 1. Verbinde dich mit MongoDB – Connection-String anpassen
client = MongoClient("mongodb://localhost:27017/")

In [6]:
db          = client["ExperimentResults20250509_182513"]
collection  = db["GPTAssistant_TransversalSkills_VolunteerDatasetSoft_qwen2.5-14b-instruct_20250509-182514_1746807914"]

# 2. Dokumente holen (Cursor vermeidet Speicher-Peak)
cursor = collection.find({}, no_cursor_timeout=True).batch_size(1_000)

# 3. In DataFrame kippen
df = pd.DataFrame(list(cursor))

# 4. BSON-Typen (z. B. ObjectId, Datumsfelder) ggf. in Strings konvertieren
df["_id"] = df["_id"].astype(str)
# Alternative: df = pd.json_normalize(json_util.loads(json_util.dumps(list(cursor))))

cursor.close()
client.close()

/opt/homebrew/Caskroom/miniconda/base/envs/iceccme_experiments/lib/python3.12/site-packages/pymongo/synchronous/collection.py:1945: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


In [9]:
df.columns

Index(['_id', 'experiment_name', 'timestamp', 'input', 'expected_output',
       'run_time_s', 'result_raw', 'skills', 'reasons', 'sentence_pieces',
       'jaccard_similarity', 'recall', 'length_penalty', 'graph_similarity',
       'combined_score', 'error', 'found_skills', 'not_found_skills',
       'accuracy', 'result_set', 'type', 'averages', 'counts'],
      dtype='object')

In [11]:
df = df[["input","expected_output","result_raw","skills","reasons","sentence_pieces", "accuracy", 'found_skills', 'not_found_skills']]
df.rename(columns={"found_skills": "in_esco", "not_found_skills": "not_in_esco"}, inplace=True)

In [14]:
# make col accuracy string
df["accuracy"] = df["accuracy"].astype(str)

In [18]:
df_y = pd.read_csv(
    "/Users/p42939_christophgassner/Code/iceccme_experiments/datasets/volunteerSoft/volunteer_softskills.csv", sep=";")
df["task_id"] = df_y["task_id"].copy()
df["short"] = df_y["short"].copy()

In [19]:
df.to_csv("gpt4.1_transversal_skills_with_short.csv", sep=";", index=False)

In [21]:
df.columns

Index(['input', 'expected_output', 'result_raw', 'skills', 'reasons',
       'sentence_pieces', 'accuracy', 'in_esco', 'not_in_esco', 'task_id',
       'short'],
      dtype='object')